In [ ]:
# Import libraries
import numpy as np
from mp_api.client import MPRester
from pymatgen.io.pwscf import PWInput
from pymatgen.core import Structure
from pymatgen.analysis.elasticity import *
from pymatgen.io.vasp.inputs import *
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import os


# Import simtool libraries
from simtool import findInstalledSimToolNotebooks,searchForSimTool
from simtool import getSimToolInputs,getSimToolOutputs,Run

# Import helper functions
from qe_functions import *

In [ ]:
# Load (or enter when prompted) your API key 
key = read_key()

In [ ]:
mp_id = "mp-8062"

In [ ]:
# Query using new Materials Project API for a specific ID
with MPRester(key, use_document_model=False) as m:
    data = m.materials.summary.search(material_ids=[mp_id])

In [ ]:
struct = data[0]['structure']
#############################################################################################################
# For some systems (ie. monoclinic) we have found using the conventional cell works better than the primitive.
# To use the conventional cell uncomment the below lines
sga = SpacegroupAnalyzer(struct)
struct = sga.get_conventional_standard_structure()
#############################################################################################################
metal = data[0]['is_metal']
n_atoms = struct.num_sites
n_atom_types = data[0]['nelements']
struct_dict = struct.as_dict()
if struct_dict['lattice']['pbc'] == (True, True, True):
    del struct_dict['lattice']['pbc']
display(struct)

In [ ]:
view_struct(struct)

In [ ]:
strains = [-0.01,-0.005,0.005,0.01]
shears = [-0.02,-0.01,0.01,0.02]

In [ ]:
# Generate strain matrices
strain_matrices = []
for ind in [(0, 0), (1, 1), (2, 2)]:
    for amount in strains:
        strain = Strain.from_index_amount(ind, amount)
        strain_matrices.append(np.array(strain))
        
for ind in [(0, 1), (0, 2), (1, 2)]:
    for amount in shears:
        strain = Strain.from_index_amount(ind, amount)
        strain_matrices.append(np.array(strain))

In [ ]:
simToolName = "elasticdft"
simToolLocation = searchForSimTool(simToolName)
for key in simToolLocation.keys():
    print("%18s = %s" % (key,simToolLocation[key]))

In [ ]:
installedSimToolNotebooks = findInstalledSimToolNotebooks(simToolName,returnString=True)
print(installedSimToolNotebooks)

In [ ]:
# get the list of inputs
inputs = getSimToolInputs(simToolLocation)
print(inputs)

In [ ]:
# get the list of outputs
outputs = getSimToolOutputs(simToolLocation)
print(outputs)

In [ ]:
#### MATERIAL INFORMATION (no need to edit) ####
inputs['mp_id'].value = mp_id
inputs['structure_dict'].value=struct_dict
inputs['num_atoms'].value=n_atoms
inputs['n_atom_types'].value=n_atom_types
inputs['space_group'].value=struct.get_space_group_info()[1]
inputs['is_metal'].value=metal
inputs['external_pressure'].value=0.0

#### Relaxation Controls ####
inputs['KE_cutoff'].value=100
inputs['k_x'].value=round((1/struct.lattice.abc[0])*(2*np.pi)/0.22)
inputs['k_y'].value=round((1/struct.lattice.abc[1])*(2*np.pi)/0.22)
inputs['k_z'].value=round((1/struct.lattice.abc[2])*(2*np.pi)/0.22)
inputs['i_steps'].value=50
inputs['e_steps'].value=200
inputs['scf_conv'].value=1e-10
inputs['energy_conv'].value=1e-10
inputs['force_conv'].value=1e-5

#### Level of Theory Controls ####
inputs['vdw_corr'].value='none'
inputs['spin_polar'].value=False
inputs['pseudo'].value='PAW'

#### HUBBARD U CONTROLS ####
# inputs['hubbard_U_on'].value=True
# inputs['hubbard_projector'].value='ortho-atomic'
# inputs['hubbard_U_values'].value={'Mg':2.0,'O':0.0001}
# inputs['hubbard_U_orbitals'].value={'Mg':'3s','O':'2p'}

#### Strain matrices ####
inputs['strain_matrices'].value=np.array(strain_matrices)
inputs['sym_red'].value=False

In [ ]:
inputs

In [ ]:
r = Run(simToolLocation,inputs)

In [ ]:
r.getResultSummary()